# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Tables](#tables)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/statics/reclassified_municipality_mapping.ipynb"
    ]
}
```

# **TABLES**

---

### STAGES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

DROP TABLE IF EXISTS [VWSSTAGE].[INHABITANTS_PER_MUNICIPALITY]
GO

CREATE TABLE [VWSSTAGE].[INHABITANTS_PER_MUNICIPALITY] (
	[ID] [INT] PRIMARY KEY IDENTITY (1,1),
	[DATE_LAST_INSERTED] [DATETIME] NOT NULL DEFAULT GETDATE(),
	[REGION] [VARCHAR](50) NOT NULL,
	[GM_CODE] [VARCHAR](50) NOT NULL,
	[VR_CODE] [VARCHAR](50) NOT NULL,
	[VR_NAME] [VARCHAR](50) NOT NULL,
	[INHABITANTS] [VARCHAR](50) NOT NULL
) ON [PRIMARY];
GO

### STATICS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY]'))
CREATE TABLE [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY] (
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE(),
    [REGIO] [VARCHAR](50) NOT NULL,
	[GMCODE] [VARCHAR](50) NOT NULL,
	[VRCODE] [VARCHAR](50) NOT NULL,
	[VRNAME] [VARCHAR](50) NOT NULL,
	[INHABITANTS] [INT] NOT NULL
);
GO

# **STORED PROCEDURES**

---

In 2022 multiple municipalties (i.e. gemeentes) are reclassified (e.g. merging) into novel muncipalities. To implement these changes, several database adjustments had to be performed.

Within the `[DBO].[SP_INHABITANTS_PER_MUNICIPALITY_STAGE_TO_STATIC]`, with help of the mapping-tabel (i.e. `[RECLASSIFIED_MUNICIPALITY_MAPPING]` ), the reclassified muncipalities will be progated towards the `[VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY]`:

```sql
LEFT JOIN (
    SELECT 
        * 
    FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
    WHERE [ACTIVE] = 1 
        AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
) B ON A.[GM_CODE]=B.[GM_CODE_ORIGINAL]
```

In turn, all downstream procedures, views or tabel - depending on `[VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY]` - will automatcally implement the modified muncipality codes (i.e. `GM_CODE` or `GMCODE`).

### STAGING &rarr; STATIC MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_INHABITANTS_PER_MUNICIPALITY_STAGE_TO_STATIC]
AS
BEGIN

    INSERT INTO [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY] (
            [REGIO], 
            [GMCODE], 
            [VRCODE], 
            [VRNAME], 
            [INHABITANTS], 
            [DATE_LAST_INSERTED]
        )
    SELECT  ISNULL(B.[GM_NAME_NEW],A.[REGION]),
            ISNULL(B.[GM_CODE_NEW],A.[GM_CODE]),
            ISNULL(B.[VR_CODE],A.[VR_CODE]),
            ISNULL(B.[VR_NAME],A.[VR_NAME]),
            CAST(ROUND(SUM(CAST(A.[INHABITANTS] AS INT) * ISNULL(B.[SHARES],1.0)),0) AS INT),
            MAX(A.[DATE_LAST_INSERTED])
    FROM [VWSSTAGE].[INHABITANTS_PER_MUNICIPALITY] A
    LEFT JOIN   (
                    SELECT 
                        * 
                    FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
                    WHERE [ACTIVE]=1
                        AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
                ) B ON A.[GM_CODE] = B.[GM_CODE_ORIGINAL]
    WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY])    
    GROUP BY ISNULL(B.[GM_NAME_NEW],A.[REGION]),
            ISNULL(B.[GM_CODE_NEW],A.[GM_CODE]),
            ISNULL(B.[VR_CODE],A.[VR_CODE]),
            ISNULL(B.[VR_NAME],A.[VR_NAME])
END;
GO

# **DATATINO CONFIGURATIONS**

---

### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'STATIC_DATA',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

-- 2) UPSERT WORKFLOW(S).....
SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @SCHEDULE = '0 12 * */3 *', -- AT 12:00 PM, EVERY 3 MONTHS
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'datafiles/STATIC/inhabitants_per_municipality.csv'
    WHEN 'acceptance' THEN 'datafiles/STATIC/inhabitants_per_municipality.csv'
    ELSE 'datafiles/STATIC/inhabitants_per_municipality.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'AzureBlob'
    WHEN 'acceptance' THEN 'AzureBlob'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'RIVM'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_name = 'SOURCE_LOAD_INHABITANTS_MUNICIPALITY_STATIC_DATA';
SET @source_description = CONCAT('LOAD: ', @workflow_name);
SET @target_name = 'VWSSTAGE.INHABITANTS_PER_MUNICIPALITY';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'REGIO|GMCODE|VRCODE|VRNAME|INHABITANTS',
    @target_columns = 'REGION|GM_CODE|VR_CODE|VR_NAME|INHABITANTS|DATE_LAST_INSERTED=GETDATE',
    @target_name = @target_name,
    @source_type = 'CsvFile',
    @location_type = @location_type,
    @delimiter_type = 'Colon',
    @security_profile= @security_profile;

-- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO INTER......
SET @source = 'dbo.SP_INHABITANTS_PER_MUNICIPALITY_STAGE_TO_STATIC';
SET @source_name = 'SOURCE_SP_INHABITANTS_PER_MUNICIPALITY_STAGE_TO_STATIC';
SET @source_description = 'MAP: INHABITANTS_PER_MUNICIPALITY FROM STAGE TO STATIC';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = 'PROCESS_LOAD_INHABITANTS_PER_MUNICIPALITY_STATIC_DATA';
SET @process_description = CONCAT('LOAD: ', @workflow_name);
SET @process_source_name = 'SOURCE_LOAD_INHABITANTS_MUNICIPALITY_STATIC_DATA';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO STATIC......
SET @process_name = 'PROCESS_SP_INHABITANTS_PER_MUNICIPALITY_STAGE_TO_STATIC';
SET @process_description = 'MAP: INHABITANTS_PER_MUNICIPALITY FROM STAGE TO STATIC';
SET @process_source_name = 'SOURCE_SP_INHABITANTS_PER_MUNICIPALITY_STAGE_TO_STATIC';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

-- 4.1) UPSERT STAGING TO STORE PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_name = 'DEPENDENCY_INHABITANTS_PER_MUNICIPALITY_STAGE_TO_STATIC';
SET @dependency_dataflow_name = 'PROCESS_SP_INHABITANTS_PER_MUNICIPALITY_STAGE_TO_STATIC';
SET @dependency_process_name = 'PROCESS_LOAD_INHABITANTS_PER_MUNICIPALITY_STATIC_DATA';
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO